In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import time
import changefont as cf

import os 
os.chdir(os.getcwd())
import sys
# insert at 1, 0 is the script path (or '' in REPL)
pydice_folder = os.path.dirname(os.getcwd())+"\\1_Model"
policy_folder = os.path.dirname(os.getcwd())+"\\5_Policy_Discovery"
sys.path.insert(1, pydice_folder)

from specify import specify_levers

from ema_workbench.analysis import parcoords
from ema_workbench.em_framework.optimization import EpsilonProgress
from ema_workbench import (save_results, load_results, Model, Policy, Scenario, 
                           Constraint, RealParameter, IntegerParameter, ScalarOutcome, 
                           MultiprocessingEvaluator, SequentialEvaluator)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
from ema_workbench.em_framework.evaluators import BaseEvaluator

ema_logging.log_to_stderr(ema_logging.INFO)
BaseEvaluator.reporting_frequency = 0.1
# ema_logging.log_to_stderr(ema_logging.DEBUG)

from PyDICE_V4 import PyDICE

In [9]:
model = PyDICE()
dice_sm = Model('dicesmEMA', function = model)

In [10]:
policy_12 = pd.read_csv("policy_12.csv", header=None, index_col=0).T

In [11]:
robust_policy = Policy('robust_policy', **specify_levers(policy_12.to_numpy()[0][0],
                                                         policy_12.to_numpy()[0][1],0,
                                                         int(policy_12.to_numpy()[0][2])))                                      

In [12]:
dice_sm.uncertainties = [IntegerParameter('t2xco2_index', 0, 999),
                         IntegerParameter('t2xco2_dist',0,2),
                         IntegerParameter('fdamage',0,2),
                         RealParameter('tfp_gr',  0.07, 0.09),
                         RealParameter('sigma_gr', -0.012, -0.008),
                         RealParameter('pop_gr', 0.1, 0.15),
                         RealParameter('fosslim',  4000.0, 13649),
                         IntegerParameter('cback', 100, 600)]

dice_sm.levers = [RealParameter('sr', 0.1, 0.5),
                  RealParameter('irstp',  0.001, 0.015),
                  IntegerParameter('periodfullpart', 10, 58),
                  IntegerParameter('miu_period', 10, 58)]

dice_sm.outcomes = [ScalarOutcome('Atmospheric Temperature 2010', ScalarOutcome.INFO),
                    ScalarOutcome('Damages 2010', ScalarOutcome.INFO),
                    ScalarOutcome('Utility 2010', ScalarOutcome.INFO),
                    ScalarOutcome('Total Output 2010', ScalarOutcome.INFO),
                    ScalarOutcome('Atmospheric Temperature 2050', ScalarOutcome.MINIMIZE),
                    ScalarOutcome('Damages 2050', ScalarOutcome.MINIMIZE),
                    ScalarOutcome('Utility 2050', ScalarOutcome.MAXIMIZE),
                    ScalarOutcome('Total Output 2050', ScalarOutcome.MAXIMIZE),
                    ScalarOutcome('Atmospheric Temperature 2100', ScalarOutcome.MINIMIZE),
                    ScalarOutcome('Damages 2100', ScalarOutcome.MINIMIZE),
                    ScalarOutcome('Utility 2100', ScalarOutcome.MAXIMIZE),
                    ScalarOutcome('Total Output 2100', ScalarOutcome.MAXIMIZE),
                    ScalarOutcome('Atmospheric Temperature 2150', ScalarOutcome.MINIMIZE),
                    ScalarOutcome('Damages 2150', ScalarOutcome.MINIMIZE),
                    ScalarOutcome('Utility 2150', ScalarOutcome.MAXIMIZE),
                    ScalarOutcome('Total Output 2150', ScalarOutcome.MAXIMIZE),
                    ScalarOutcome('Atmospheric Temperature 2200', ScalarOutcome.MINIMIZE),
                    ScalarOutcome('Damages 2200', ScalarOutcome.MINIMIZE),
                    ScalarOutcome('Utility 2200', ScalarOutcome.MAXIMIZE),
                    ScalarOutcome('Total Output 2200', ScalarOutcome.MAXIMIZE),
                    ScalarOutcome('Atmospheric Temperature 2300', ScalarOutcome.MINIMIZE),
                    ScalarOutcome('Damages 2300', ScalarOutcome.MINIMIZE),
                    ScalarOutcome('Utility 2300', ScalarOutcome.MAXIMIZE),
                    ScalarOutcome('Total Output 2300', ScalarOutcome.MAXIMIZE)]

In [13]:
n_scenarios=540000
n_policies=robust_policy

start = time.time()
with MultiprocessingEvaluator(dice_sm) as evaluator:
    results = evaluator.perform_experiments(scenarios=n_scenarios, policies=n_policies)
end = time.time()

print('Experiment time is ' + str(round((end - start)/60)) + ' mintues')

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 540000 scenarios * 1 policies * 1 model(s) = 540000 experiments
[MainProcess/INFO] 54000 cases completed
[MainProcess/INFO] 108000 cases completed
[MainProcess/INFO] 162000 cases completed
[MainProcess/INFO] 216000 cases completed
[MainProcess/INFO] 270000 cases completed
[MainProcess/INFO] 324000 cases completed
[MainProcess/INFO] 378000 cases completed
[MainProcess/INFO] 432000 cases completed
[MainProcess/INFO] 486000 cases completed
[MainProcess/INFO] 540000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


Experiment time is 16 mintues


In [14]:
experiments, outcomes = results

In [16]:
pd.DataFrame(outcomes).describe()

,Atmospheric Temperature 2010,Damages 2010,Utility 2010,Total Output 2010,Atmospheric Temperature 2050,Damages 2050,Utility 2050,Total Output 2050,Atmospheric Temperature 2100,Damages 2100,...,Utility 2150,Total Output 2150,Atmospheric Temperature 2200,Damages 2200,Utility 2200,Total Output 2200,Atmospheric Temperature 2300,Damages 2300,Utility 2300,Total Output 2300
count,5.400000e+05,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,...,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.0,540000.000000,540000.000000,540000.000000
mean,8.000000e-01,0.067059,-3698.461713,63.514692,1.730159,1.350727,-1261.883517,200.302588,1.624634,3.461033,...,9044.585921,1013.260193,0.004527,0.014598,14507.764826,1622.702023,0.0,0.007995,24398.685455,2969.921071
std,7.486130e-12,0.047497,0.177969,0.047497,0.423950,1.164936,38.897623,5.522842,0.639797,4.010789,...,172.842341,12.745545,0.059223,0.252976,190.846998,9.782684,0.0,0.011307,199.295506,5.208259
min,8.000000e-01,0.000584,-3698.618207,63.472934,0.191857,0.001626,-1362.286786,182.553412,0.061873,0.001503,...,8445.979885,916.577054,0.000000,0.000000,13839.641104,1575.055405,0.0,0.000000,23707.112570,2951.403795
25%,8.000000e-01,0.000584,-3698.617258,63.473187,1.560856,0.105472,-1293.622689,196.370484,1.246462,0.199909,...,8911.485838,1004.451595,0.000000,0.000000,14360.979241,1615.977961,0.0,0.000000,24246.017932,2966.703076
50%,8.000000e-01,0.091945,-3698.554921,63.489806,1.809143,1.423650,-1261.077611,200.277854,1.651146,2.396175,...,9055.372444,1013.805220,0.000000,0.000000,14522.000652,1623.391115,0.0,0.000000,24414.993706,2970.696947
75%,8.000000e-01,0.108649,-3698.212958,63.581083,2.004975,2.233582,-1230.038302,204.266574,2.032263,5.309903,...,9180.724693,1022.844113,0.000000,0.013157,14658.599946,1630.296314,0.0,0.023976,24556.183143,2974.004479
max,8.000000e-01,0.108649,-3698.212011,63.581336,2.767369,6.371924,-1172.711566,215.007836,4.090624,40.352152,...,9435.962230,1041.730479,2.207086,27.036879,14931.696786,1642.776541,0.0,0.024016,24837.320486,2979.063397
